In [1]:
# Aluno: Lucas Barbosa Bulcão Mota

In [2]:
!pip install conllu
!wget http://marlovss.work.gd:8080/tomorrow/aula2/bosque.conllu

'wget' n�o � reconhecido como um comando interno
ou externo, um programa oper�vel ou um arquivo em lotes.


In [3]:
import conllu
import itertools as it

class AttributeDict(dict):
    __getattr__ = dict.__getitem__
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__


class CoNLLU:
   def __init__(self, files):
      self.words = []
      self.sentences = []
      for f in files:
         parsed = conllu.parse(open(f).read())
         sents = [[AttributeDict(form = token['form'], lemma=token['lemma'],pos=token['upos'],feats=token['feats']) for token in tokenlist if token['upos']!='_'] for tokenlist in parsed]
         self.sentences.extend(sents)
         self.words.extend([word for sent in sents for word in sent])
      self.pos_tags = set([word.pos for word in self.words])
      self.feats_dict ={pos:set(it.chain.from_iterable([list(word.feats.keys()) for word in self.words if word.pos==pos and word.feats!= None])) for pos in self.pos_tags}


ModuleNotFoundError: No module named 'conllu'

In [ ]:
bosque = CoNLLU(files=["bosque.conllu"])

In [ ]:
# Função de acurácia para TAGGER
# Não desenvolvi uma função para analisar as flexões porque cada classe gramatical tem uma quantidade de campos diferentes

def accuracy(predicted,gold):
   acertos = len([predicted[i][j][1] for i in range(len(gold)) for j in range(len(gold[i])) if predicted[i][j][1]==gold[i][j][1]])
   totais = sum([len(sent) for sent in gold])
   return acertos/totais

In [ ]:
# Listas com diversas palavras que serão utilizadas nas funçoes de tag e flexão

ADP = [ "a", "ante", "após", "até", "com", "contra", "de", "desde", "em","entre", "para", "per", "perante", "por", "sem", "sob", "sobre", "trás"]
pronomes = {
        'eu': {'genero': 'invariável', 'numero': 'singular', 'tipo': 'pessoal'},
        'tu': {'genero': 'invariável', 'numero': 'singular', 'tipo': 'pessoal'},
        'ele': {'genero': 'masculino', 'numero': 'singular', 'tipo': 'pessoal'},
        'ela': {'genero': 'feminino', 'numero': 'singular', 'tipo': 'pessoal'},
        'nós': {'genero': 'invariável', 'numero': 'plural', 'tipo': 'pessoal'},
        'vós': {'genero': 'invariável', 'numero': 'plural', 'tipo': 'pessoal'},
        'eles': {'genero': 'masculino', 'numero': 'plural', 'tipo': 'pessoal'},
        'elas': {'genero': 'feminino', 'numero': 'plural', 'tipo': 'pessoal'},
        'meu': {'genero': 'masculino', 'numero': 'singular', 'tipo': 'possessivo'},
        'minha': {'genero': 'feminino', 'numero': 'singular', 'tipo': 'possessivo'},
        'meus': {'genero': 'masculino', 'numero': 'plural', 'tipo': 'possessivo'},
        'minhas': {'genero': 'feminino', 'numero': 'plural', 'tipo': 'possessivo'},
        'teu': {'genero': 'masculino', 'numero': 'singular', 'tipo': 'possessivo'},
        'tua': {'genero': 'feminino', 'numero': 'singular', 'tipo': 'possessivo'},
        'teus': {'genero': 'masculino', 'numero': 'plural', 'tipo': 'possessivo'},
        'tuas': {'genero': 'feminino', 'numero': 'plural', 'tipo': 'possessivo'},
        'seu': {'genero': 'masculino', 'numero': 'singular', 'tipo': 'possessivo'},
        'sua': {'genero': 'feminino', 'numero': 'singular', 'tipo': 'possessivo'},
        'seus': {'genero': 'masculino', 'numero': 'plural', 'tipo': 'possessivo'},
        'suas': {'genero': 'feminino', 'numero': 'plural', 'tipo': 'possessivo'},
        'este': {'genero': 'masculino', 'numero': 'singular', 'tipo': 'demonstrativo'},
        'esta': {'genero': 'feminino', 'numero': 'singular', 'tipo': 'demonstrativo'},
        'estes': {'genero': 'masculino', 'numero': 'plural', 'tipo': 'demonstrativo'},
        'estas': {'genero': 'feminino', 'numero': 'plural', 'tipo': 'demonstrativo'},
        'isso': {'genero': 'neutro', 'numero': 'singular', 'tipo': 'demonstrativo'},
        'aquilo': {'genero': 'neutro', 'numero': 'singular', 'tipo': 'demonstrativo'},
        }

Artigos = {"o","a","os","as","um","uma","uns","umas"}
verbos_infinitivos = ['ar', 'er', 'ir']
sufixos = {
    'presente_indicativo':['o', 'as', 'a', 'amos', 'ais', 'am','o', 'es', 'e', 'emos', 'eis', 'em','o', 'es', 'e', 'imos', 'is', 'em'],
    'pretérito_perfeito_indicativo':['ei', 'aste', 'ou', 'amos', 'astes', 'aram','i', 'este', 'eu', 'emos', 'estes', 'eram','i', 'iste', 'iu', 'imos', 'istes','iram'],
    'pretérito_imperfeito_indicativo':['ava', 'avas', 'ava', 'ávamos', 'áveis', 'avam','ia', 'ias', 'ia', 'íamos', 'íeis', 'iam','ia', 'ias', 'ia', 'íamos', 'íeis', 'iam'],
    'pretérito_mais_que_perfeito_indicativo': ['ara', 'aras', 'ara', 'áramos', 'áreis', 'aram','era', 'eras', 'era', 'êramos', 'êreis', 'eram','ira', 'iras', 'ira', 'íramos', 'íreis', 'iram'],
    'futuro_do_presente_indicativo':['arei', 'arás', 'ará', 'aremos', 'areis', 'arão','erei', 'erás', 'erá', 'eremos', 'ereis', 'erão','irei', 'irás', 'irá', 'iremos', 'ireis', 'irão'],
    'futuro_do_pretérito_indicativo':['aria', 'arias', 'aria', 'aríamos', 'aríeis', 'ariam','eria', 'erias', 'eria', 'eríamos', 'eríeis', 'eriam','iria', 'irias', 'iria', 'iríamos', 'iríeis', 'iriam'],
    'presente_subjuntivo': ['e', 'es', 'e', 'emos', 'eis', 'em','a', 'as', 'a', 'amos', 'ais', 'am','a', 'as', 'a', 'amos', 'ais', 'am'],
    'pretérito_imperfeito_subjuntivo': ['asse', 'asses', 'asse', 'ássemos', 'ásseis', 'assem','esse', 'esses', 'esse', 'êssemos', 'êsseis', 'essem','isse', 'isses', 'isse', 'íssemos', 'ísseis', 'issem'],
    'futuro_subjuntivo':['ar', 'ares', 'ar', 'armos', 'ardes', 'arem','er', 'eres', 'er', 'ermos', 'erdes', 'erem','ir', 'ires', 'ir', 'irmos', 'irdes', 'irem'],
}
substantivos_singular_com_s = ['país','lápis','cais','óculos','ônibus','vírus','pires']
masculinos = ["o", "os","or", "e", "m", "l", "r", "u", "z", "n","ei"]
femininos = ["a","as", "ção", "ade", "ice", "gem", "ez", "iz", "ssa", "ina", "ice", "ude","ite"]
excecoes_masculinas = [
        "dia", "mapa", "planeta", "problema", "sistema", "tema",
        "clima", "poema", "drama", "dilema", "fonema", "estratagema",
        "protoplasma", "grama", "buda"
    ]
excecoes_femininas = [
        "mão", "coração", "leite", "peixe", "repolho", "cúspide",
        "alface", "faringe", "bronquite", "polenta", "tosse",
        "consciência", "água", "paz"
    ]


In [ ]:
# Suf_to_tag é um Dicionario que relaciona os sufixos mais comuns no CORPUS e suas classes gramaticais
# demora aproximadamente 15 minutos para carregar

from nltk.probability import FreqDist

suffixes = set([word.form.lower()[-4:] for word in bosque.words])
suf_to_tag = {suf:FreqDist([word.pos for word in bosque.words if word.form.lower()[-4:]==suf]).max() for suf in suffixes}

In [ ]:
def tag(tokens):
  tagged = []

  for token in tokens:
    if token.lower() in Artigos:
      tagged.append((token.lower(), "DET"))
    elif token.lower() in ADP:
      tagged.append((token.lower(), "ADP"))
    elif token.lower()[-4:] in suf_to_tag:
      tagged.append((token.lower(), suf_to_tag[token.lower()[-4:]]))
    elif token.lower()[-3:] in suf_to_tag:
      tagged.append((token.lower(), suf_to_tag[token.lower()[-3:]]))
    elif token.lower()[-2:] in suf_to_tag:
      tagged.append((token.lower(), suf_to_tag[token.lower()[-2:]]))
    elif token.lower()[-1:] in suf_to_tag:
      tagged.append((token.lower(), suf_to_tag[token.lower()[-1:]]))
    else:
      tagged.append((token.lower(),"_"))

  return tagged

In [ ]:
# Teste do POS TAGGER

x = tag(["o","rei","é","bonito"])
y = tag(["o","rato","roeu","a","roupa","do","rei","de","roma"])
print(x)
print(y)

[('o', 'DET'), ('rei', 'NOUN'), ('é', 'AUX'), ('bonito', 'ADJ')]
[('o', 'DET'), ('rato', 'NOUN'), ('roeu', 'PRON'), ('a', 'DET'), ('roupa', 'NOUN'), ('do', 'PROPN'), ('rei', 'NOUN'), ('de', 'ADP'), ('roma', 'PROPN')]


In [ ]:
!wget http://marlovss.work.gd:8080/tomorrow/aula2/test.conllu

--2024-08-03 02:03:22--  http://marlovss.work.gd:8080/tomorrow/aula2/test.conllu
Resolving marlovss.work.gd (marlovss.work.gd)... 177.180.149.154
Connecting to marlovss.work.gd (marlovss.work.gd)|177.180.149.154|:8080... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1819980 (1.7M)
Saving to: ‘test.conllu.2’

test.conllu.2       100%[===================>]   1.74M   404KB/s    in 4.9s    

2024-08-03 02:03:28 (360 KB/s) - ‘test.conllu.2’ saved [1819980/1819980]



In [ ]:
# Acurácia do POS TAGGER = 83%

test = CoNLLU(files=["test.conllu"])
test_sents = [[word.form for word in sent] for sent in test.sentences]
gold = [[(word.form.lower(),word.pos) for word in sent] for sent in test.sentences]
predicted = [tag(sent) for sent in test_sents]

accuracy(predicted,gold)

0.8367627880017389

In [ ]:
  # Função que flexiona cada palavra da sentença

def flexoes(tagged):

    for i, (palavra, tag) in enumerate(tagged):
      temp_list = list(tagged[i])

      if tag == "NOUN": # Uma ideia seria identificar o artigo e com base nele flexionar o substantivo. Devido ao tempo não consegui desenvolver o código da melhor forma
        numero = "_"
        genero = "_"
        if palavra.lower() in substantivos_singular_com_s:
          numero = "singular"
          genero = "masculino"
        else:
          if palavra.endswith("s"):
            numero = "plural"
          else:
            numero = "singular"
          if palavra.lower() in excecoes_masculinas:
            genero = "masculino"
          elif palavra.lower() in excecoes_femininas:
            genero = "feminino"
          else:
            for g in masculinos:
              if palavra.endswith(g):
                genero = "masculino"
            else:
              for g in femininos:
                if palavra.endswith(g):
                  genero = "feminino"
        temp_list.append(genero)
        temp_list.append(numero)
        tagged[i] = tuple(temp_list)

      elif tag == "ADJ":
        if palavra.endswith('a'):
          genero = 'feminino'
          numero = 'singular'
        elif palavra.endswith('as'):
          genero = 'feminino'
          numero = 'plural'
        elif palavra.endswith('o'):
            genero = 'masculino'
            numero = 'singular'
        elif palavra.endswith('os'):
            genero = 'masculino'
            numero = 'plural'
        elif palavra.endswith('e'):
            genero = 'invariável'
            numero = 'singular'
        elif palavra.endswith('es'):
            genero = 'invariável'
            numero = 'plural'
        elif palavra.endswith('l'):
            genero = 'invariável'
            numero = 'singular'
        elif palavra.endswith('is'):
            genero = 'invariável'
            numero = 'plural'
        elif palavra.endswith('ista'):
            genero = 'invariável'
            numero = 'singular'
        elif palavra.endswith('istas'):
            genero = 'invariável'
            numero = 'plural'
        else:
            genero = '_'
            numero = '_'
        temp_list.append(genero)
        temp_list.append(numero)
        tagged[i] = tuple(temp_list)

      elif tag == "VERB":
        if palavra.lower()[-2:] in verbos_infinitivos:
          temp_list.append(palavra)
          temp_list.append("INFINITIVO")
          tagged[i] = tuple(temp_list)
        else:
          aux = 0
          for tempo, terminacao in sufixos.items():
            for aux in terminacao:
              if palavra.endswith(aux) and aux != 1:
                temp_list.append(palavra)
                temp_list.append(tempo)
                tagged[i] = tuple(temp_list)
                aux = 1
                break
      elif tag == "PRON":
        if palavra in pronomes:
          temp_list.append(pronomes[palavra]["genero"])
          temp_list.append(pronomes[palavra]["numero"])
          temp_list.append(pronomes[palavra]["tipo"])
          tagged[i] = tuple(temp_list)
        else:
          temp_list.append("_")
          temp_list.append("_")
          temp_list.append("_")
          tagged[i] = tuple(temp_list)
      elif tag == "DET":
        if palavra == "o":
          temp_list.append("masculino")
          temp_list.append("singular")
          tagged[i] = tuple(temp_list)
        elif palavra == "os":
          temp_list.append("masculino")
          temp_list.append("plural")
          tagged[i] = tuple(temp_list)
        elif palavra == "a":
          temp_list.append("feminino")
          temp_list.append("singular")
          tagged[i] = tuple(temp_list)
        elif palavra == "as":
          temp_list.append("feminino")
          temp_list.append("plural")
          tagged[i] = tuple(temp_list)
        elif palavra == "um":
          temp_list.append("masculino")
          temp_list.append("singular")
          tagged[i] = tuple(temp_list)
        elif palavra == "uma":
          temp_list.append("feminino")
          temp_list.append("singular")
          tagged[i] = tuple(temp_list)
        elif palavra == "uns":
          temp_list.append("masculino")
          temp_list.append("plural")
          tagged[i] = tuple(temp_list)
        elif palavra == "umas":
          temp_list.append("feminino")
          temp_list.append("plural")
          tagged[i] = tuple(temp_list)

      else:
        temp_list.append(palavra)
        temp_list.append("_")
        tagged[i] = tuple(temp_list)

    return tagged

In [ ]:
# Função de Classificação mas agora também chama a função de flexão

def tag(tokens):
  tagged = []

  for token in tokens:
    if token.lower() in Artigos:
      tagged.append((token.lower(), "DET"))
    elif token.lower() in ADP:
      tagged.append((token.lower(), "ADP"))
    elif token.lower()[-4:] in suf_to_tag:
      tagged.append((token.lower(), suf_to_tag[token.lower()[-4:]]))
    elif token.lower()[-3:] in suf_to_tag:
      tagged.append((token.lower(), suf_to_tag[token.lower()[-3:]]))
    elif token.lower()[-2:] in suf_to_tag:
      tagged.append((token.lower(), suf_to_tag[token.lower()[-2:]]))
    elif token.lower()[-1:] in suf_to_tag:
      tagged.append((token.lower(), suf_to_tag[token.lower()[-1:]]))
    else:
      tagged.append((token.lower(),"_"))

  flexoes(tagged)
  return tagged

In [ ]:
# sentenças de Exemplo da função de POS TAGGING + FLEXÃO (NOUN, ADJ, VERB, PRON, DET)

sentencas = [
    ['Eu', 'gosto', 'de', 'estudar', 'programação'],
    ['Ela', 'foi', 'ao', 'mercado', 'comprar', 'frutas'],
    ['O', 'cachorro', 'correu', 'pelo', 'parque', 'rapidamente'],
    ['Nós', 'vamos', 'viajar', 'para', 'a', 'praia', 'no', 'próximo', 'fim', 'de', 'semana'],
    ['Eles', 'estavam', 'muito', 'felizes', 'com', 'a', 'notícia'],
    ['A', 'aula', 'de', 'matemática', 'é', 'muito', 'interessante'],
    ['João', 'e', 'Maria', 'foram', 'ao', 'cinema', 'ontem'],
    ['O', 'sol', 'estava', 'brilhando', 'intensamente', 'no', 'céu'],
    ['Eu', 'preciso', 'comprar', 'um', 'presente', 'para', 'o', 'aniversário', 'da', 'Ana'],
    ['O', 'livro', 'que', 'estou', 'lendo', 'é', 'muito', 'interessante']
]

for sentenca in sentencas:
    print(tag(sentenca))

[('eu', 'PRON', 'invariável', 'singular', 'pessoal'), ('gosto', 'NOUN', 'masculino', 'singular'), ('de', 'ADP', 'de', '_'), ('estudar', 'VERB', 'estudar', 'INFINITIVO'), ('programação', 'NOUN', 'feminino', 'singular')]
[('ela', 'PRON', 'feminino', 'singular', 'pessoal'), ('foi', 'AUX', 'foi', '_'), ('ao', 'ADP', 'ao', '_'), ('mercado', 'NOUN', 'masculino', 'singular'), ('comprar', 'VERB', 'comprar', 'INFINITIVO'), ('frutas', 'NOUN', 'feminino', 'plural')]
[('o', 'DET', 'masculino', 'singular'), ('cachorro', 'NOUN', 'masculino', 'singular'), ('correu', 'VERB', 'correu', 'pretérito_perfeito_indicativo'), ('pelo', 'NOUN', 'masculino', 'singular'), ('parque', 'SCONJ', 'parque', '_'), ('rapidamente', 'ADV', 'rapidamente', '_')]
[('nós', 'PRON', 'invariável', 'plural', 'pessoal'), ('vamos', 'VERB', 'vamos', 'presente_indicativo', 'vamos', 'pretérito_perfeito_indicativo', 'vamos', 'presente_subjuntivo'), ('viajar', 'VERB', 'viajar', 'INFINITIVO'), ('para', 'ADP', 'para', '_'), ('a', 'DET', 'f